In [22]:
# =========================
# 📌 Install dependencies
# =========================
!pip install gspread oauth2client moviepy gTTS pydub

import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from moviepy.editor import VideoFileClip, concatenate_videoclips, AudioFileClip, CompositeAudioClip
from gtts import gTTS
import requests

# =========================
# 📌 Google Drive Mount
# =========================
from google.colab import drive
drive.mount('/content/drive')

# Folder in Google Drive
DRIVE_FOLDER = "/content/drive/MyDrive/ColabVideos"
os.makedirs(DRIVE_FOLDER, exist_ok=True)

# =========================
# 📌 Google Sheets Auth
# =========================
SCOPE = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]

CREDS_FILE = "service_account.json"  # Upload this JSON to Colab

creds = ServiceAccountCredentials.from_json_keyfile_name(CREDS_FILE, SCOPE)
client = gspread.authorize(creds)

SPREADSHEET_ID = "1RZU0zDru5xZ8pbac7BhZnIdNBoRDprgwt9XsXEhEi4s"
sheet = client.open_by_key(SPREADSHEET_ID).sheet1
print("✅ Google Sheet authenticated successfully!")

# =========================
# 📌 Video Creation Function
# =========================
def create_video(topic, row):
    if not topic.strip():
        print(f"⚠ Skipping empty topic at row {row}")
        return

    print(f"🎬 Creating video for: {topic}")

    # 1️⃣ Download a stock video
    url = "https://cdn.pixabay.com/vimeo/370826595/pexels-video-370826595.mp4"
    video_filename = f"clip_{row}.mp4"
    try:
        r = requests.get(url, stream=True, timeout=10)
        with open(video_filename, "wb") as f:
            for chunk in r.iter_content(chunk_size=1024):
                f.write(chunk)
    except Exception as e:
        print(f"❌ Failed to download video: {e}")
        return

    # 2️⃣ Convert text to speech
    tts_filename = f"narration_{row}.mp3"
    tts = gTTS(text=topic, lang="en")
    tts.save(tts_filename)

    # 3️⃣ Load video + audio
    video_clip = VideoFileClip(video_filename).subclip(0, 10)  # first 10 sec
    narration = AudioFileClip(tts_filename)

    # Optional BGM
    bgm = AudioFileClip("https://www.soundhelix.com/examples/mp3/SoundHelix-Song-1.mp3").volumex(0.2)

    # 4️⃣ Merge narration + BGM
    final_audio = CompositeAudioClip([narration.volumex(1.0), bgm.set_duration(video_clip.duration)])
    final_clip = video_clip.set_audio(final_audio)

    # 5️⃣ Export video to Google Drive
    output_file = os.path.join(DRIVE_FOLDER, f"final_video_{row}.mp4")
    final_clip.write_videofile(output_file, codec="libx264", audio_codec="aac")

    # ✅ Update sheet with status
    sheet.update_cell(row, 2, "✅ Saved to Drive")
    print(f"🎉 Video saved at: {output_file}")

# =========================
# 📌 Main Execution
# =========================
all_topics = sheet.col_values(1)[1:]  # skip header row
for row, topic in enumerate(all_topics, start=2):
    create_video(topic, row)

print("✅ All videos generated and saved to Google Drive!")


SpreadsheetNotFound: <Response [200]>